# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [95]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [96]:
# Read the Mapbox API key
mapbox_token = os.getenv("MAPBOX_API_KEY")
load_dotenv()

True

# Import Data

In [97]:
# Import the CSVs to Pandas DataFrames
File_Path = Path("Data/sfo_neighborhoods_census_data.csv")
SFO_Data = pd.read_csv(File_Path, index_col="year")

File_Path = Path("Data/neighborhoods_coordinates.csv")
DF_Neighborhood_Locations = pd.read_csv(File_Path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [98]:
# Define Panel Visualization Functions
def Housing_Units_Per_Year():
    """Housing Units Per Year."""
    Mean_Housing_Units = SFO_Data.groupby(["year"])["housing_units"].mean()
    SFO_Grouped_MIN = SFO_Data.groupby(["year"])["housing_units"].min().min()
    SFO_Grouped_MAX = SFO_Data.groupby(["year"])["housing_units"].max().max()
    SFO_Grouped_STD = SFO_Data["housing_units"].std()
    #TODO fix years...
    First_Year = Mean_Housing_Units.index[0]
    Last_Year = Mean_Housing_Units.index[-1]
    Fig_Housing_Units = px.bar(Mean_Housing_Units,
                               title= "Housing Units from 2010 to 2016 in San Francisco")
    return Fig_Housing_Units

def Average_Gross_Rent():
    """Average Gross Rent in San Francisco Per Year."""
    SFO_Grouped_by_Year = SFO_Data.groupby(["year"])
    Mean_Gross_Rent = SFO_Grouped_by_Year["gross_rent"].mean()
    Mean_Gross_Rent_Plot = px.line(Mean_Gross_Rent,
                                   title ="Average Gross Rent in San Francisco")
    return Mean_Gross_Rent_Plot

def Average_Sales_Price():
    """Average Sales Price Per Year."""
    SFO_Grouped_by_Year = SFO_Data.groupby(["year"])
    Mean_Price_per_SQFT = SFO_Grouped_by_Year["sale_price_sqr_foot"].mean()
    Mean_Price_per_SQFT_Plot = px.line(Mean_Price_per_SQFT,
                                       title = "Average Sales Price per Year in San Francisco")
    return Mean_Price_per_SQFT_Plot

def Average_Price_by_Neighborhood():
    """Average Prices by Neighborhood."""
    SFO_Neighborhood = pd.read_csv("Data/sfo_neighborhoods_census_data.csv")
    SFO_Grouped_by_Year_and_NBHD = SFO_Neighborhood.groupby(["year", "neighborhood"])
    SFO_Grouped_by_Year_and_NBHD.head()
    Avg_Price_Mean = SFO_Grouped_by_Year_and_NBHD.mean()
    SFO_Grouped_Year_and_NBHD_Plot = Avg_Price_Mean.hvplot.line(
        x='year',
        y='sale_price_sqr_foot',
        xlabel='Year',
        ylabel='Price per Sqr Foot',
        groupby='neighborhood',
        width=600
        ).opts(yformatter='%0f')
    return SFO_Grouped_Year_and_NBHD_Plot

def TOP_Expensive_Neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    Groupby_Neighborhood = SFO_Data.groupby(["neighborhood"]).mean()
    TOP_10_Expensive = Groupby_Neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    TOP_10_Expensive = TOP_10_Expensive.reset_index()
    TOP_10_Expensive_Plot = TOP_10_Expensive.hvplot.bar(
        x="neighborhood",
        y="sale_price_sqr_foot",
        xlabel = "Neighborhoods",
        ylabel = "Average Sale Price Per Square Foot",
        title="Top 10 Most Expensive Neighborhoods in SFO",
        height=400,
        rot=90)
    return TOP_10_Expensive_Plot

def Parallel_Coordinates():
    """Parallel Coordinates Plot."""
    Groupby_Neighborhood = SFO_Data.groupby(["neighborhood"]).mean()
    TOP_10_Expensive_1 = Groupby_Neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    TOP_10_Expensive_1 = TOP_10_Expensive_1.reset_index()
    TOP_10_Expensive_Parallel = px.parallel_coordinates(TOP_10_Expensive_1, color='sale_price_sqr_foot')
    return TOP_10_Expensive_Parallel

def Parallel_Categories():
    """Parallel Categories Plot."""
    Groupby_Neighborhood = SFO_Data.groupby(["neighborhood"]).mean()
    TOP_10_Expensive_2 = Groupby_Neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    TOP_10_Expensive_2 = TOP_10_Expensive_2.reset_index()
    TOP_10_Expensive_Categories =  px.parallel_categories(
            TOP_10_Expensive_2,
            dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
            color="sale_price_sqr_foot",
            color_continuous_scale=px.colors.sequential.Inferno,
        )
    return TOP_10_Expensive_Categories

def Neighborhood_Map():
    """Neighborhood Map"""
    Mean_Data = SFO_Data.groupby(["neighborhood"]).mean()
    Mean_Data.reset_index(inplace=True)
    Combined_Data = pd.concat([DF_Neighborhood_Locations, Mean_Data], axis='columns', join='inner')
    Combined_Data = Combined_Data.iloc[:, 1:]
    px.set_mapbox_access_token(mapbox_token)
    Neighborhood_Map_Plot = px.scatter_mapbox(
            Combined_Data,
            lat="Lat",
            lon="Lon",
            size="gross_rent",
            color="neighborhood"
        )
    return Neighborhood_Map_Plot

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [101]:
Housing_Units = pn.Column(
    Average_Sales_Price(),
    Average_Gross_Rent(),
    Housing_Units_Per_Year())

Housing_Avg = pn.Column(
    Average_Price_by_Neighborhood(),
    TOP_Expensive_Neighborhoods())

Parallel_Plots = pn.Column(
    Parallel_Coordinates(),
    Parallel_Categories())

Map_Plot = pn.Column(
    Neighborhood_Map())

Dashboard = pn.Tabs(
                ("Welcome", Map_Plot),
                ("Yearly Market Analysis", Housing_Units),
                ("Neighborhood Analysis", Housing_Avg),
                ("Parallel Plots", Parallel_Plots))        
Dashboard        

Tabs
    [0] Column
        [0] Plotly(Figure)
    [1] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [1] HoloViews(Bars)
    [3] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)

## Serve the Panel Dashboard


In [100]:
Dashboard.servable()

Tabs
    [0] Column
        [0] Plotly(Figure)
    [1] Column
        [0] Plotly(Figure, viewport={'xaxis.range': [2010, ...})
        [1] Plotly(Figure, viewport={'xaxis.range': [2010, ...})
        [2] Plotly(Figure, viewport={'xaxis.range': [2009.5, ...})
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [1] HoloViews(Bars)
    [3] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)